In [169]:
import pandas as pd
# 서울교통공사 제공 위경도 정보 기준 정리
latlon_df = pd.read_csv("서울교통공사_1_8호선 역사 좌표(위경도) 정보_20241031.csv", encoding="cp949")
latlon_df["역명"] = latlon_df["역명"].str.strip()
latlon_df["노드명"] = latlon_df["역명"] + "_" + latlon_df["호선"].astype(str)
latlon_df.head()



,연번,호선,고유역번호(외부역코드),역명,위도,경도,작성일자,노드명
0,1,1,150,서울,37.553150,126.972533,1974-02-28,서울_1
1,2,1,151,시청,37.563590,126.975407,1974-08-15,시청_1
2,3,1,152,종각,37.570203,126.983116,1974-08-15,종각_1
3,4,1,153,종로3가,37.570429,126.992095,1974-08-15,종로3가_1
4,5,1,154,종로5가,37.570971,127.001900,1974-03-31,종로5가_1


In [170]:
distance_df = pd.read_csv("국가철도공단_수도권4호선_역간거리_20230516.csv", encoding="cp949")
speed_df = pd.read_csv("서울교통공사_1_8호선 표정속도 정보_20250314.csv", encoding="cp949")

# 4호선 기준 표정속도 추출
# "4호선"이 아니라 숫자 4가 들어가 있는지 확인 후 처리
if 4 in speed_df["호선"].values:
    line_speed = speed_df.loc[speed_df["호선"] == 4, "표정속도"].iloc[0]
    print("4호선 표정속도:", line_speed)
else:
    print("4호선 데이터가 표정속도 테이블에 없습니다.")

# 소요시간 계산
distance_df["예상소요시간"] = (distance_df["역간거리(km)"] / line_speed) * 60
distance_df.head()


4호선 표정속도: 40.1


,철도운영기관명,선명,역명,역간거리(km),예상소요시간
0,코레일,4호선,대공원,1.0,1.496259
1,코레일,4호선,과천,1.0,1.496259
2,코레일,4호선,인덕원,3.0,4.488778
3,코레일,4호선,범계,2.6,3.890274
4,코레일,4호선,대야미,2.6,3.890274


In [172]:
transfer_df = pd.read_csv("서울교통공사_서울 도시철도 환승정보_20250319.csv", encoding="cp949")
# 환승역끼리 그룹핑하여 각 역마다 환승 대상 호선을 추출
grouped = transfer_df.groupby("환승시작역")["환승종료 호선"].unique().reset_index()
grouped.columns = ["역명", "환승호선리스트"]

# 각 역별 transfer1, transfer2로 나눠 저장
grouped["transfer1"] = grouped["환승호선리스트"].apply(lambda x: x[0] if len(x) > 0 else "")
grouped["transfer2"] = grouped["환승호선리스트"].apply(lambda x: x[1] if len(x) > 1 else "")
transfer_df = grouped[["역명", "transfer1", "transfer2"]]
transfer_df.head(10)

,역명,transfer1,transfer2
0,가락시장,8,3
1,가산디지털단지,7,1
2,가좌,경의선,
3,강남,2,
4,강남구청,7,수인분당선
5,강동,5,
6,건대입구,7,2
7,계양,공항철도,인천1
8,고속터미널,9,7
9,고잔,수인분당선,4


In [174]:
!pip install matplotlib-venn

In [175]:
!pip install ace_tools

In [176]:
import pandas as pd

# 위경도 정보 로드
latlon_df = pd.read_csv("/content/서울교통공사_1_8호선 역사 좌표(위경도) 정보_20241031.csv", encoding="cp949")
latlon_df["역명"] = latlon_df["역명"].str.strip()
latlon_df["노드명"] = latlon_df["역명"] + "_" + latlon_df["호선"].astype(str)
latlon_df.rename(columns={"위도": "cy", "경도": "cx"}, inplace=True)

# 환승 정보 로드 및 정제
transfer_df = pd.read_csv("/content/서울교통공사_서울 도시철도 환승정보_20250319.csv", encoding="cp949")
transfer_grouped = transfer_df.groupby("환승시작역")["환승종료 호선"].unique().reset_index()
transfer_grouped["transfer1"] = transfer_grouped["환승종료 호선"].apply(lambda x: x[0] if len(x) > 0 else "")
transfer_grouped["transfer2"] = transfer_grouped["환승종료 호선"].apply(lambda x: x[1] if len(x) > 1 else "")
transfer_grouped.rename(columns={"환승시작역": "역명"}, inplace=True)

# 표정속도 정보
speed_df = pd.read_csv("/content/서울교통공사_1_8호선 표정속도 정보_20250314.csv", encoding="cp949")
speed_df = speed_df[speed_df["호선"].apply(lambda x: str(x).isdigit())]
speed_df["호선"] = speed_df["호선"].astype(int)
speed_dict = speed_df.groupby("호선")["표정속도"].mean().to_dict()

# 유연한 역간 거리 정보 로드
def load_distance_data(filepath):
    df = pd.read_csv(filepath, encoding="cp949")
    for colname in ["역간거리(km)", "역간 거리(km)", "역간거리"]:
        if colname in df.columns:
            df = df[["선명", "역명", colname]]
            df.columns = ["line", "station", "distance_km"]
            return df
    raise ValueError(f"지원되지 않는 컬럼 구조: {filepath}")

distance_dfs = [
    load_distance_data("/content/국가철도공단_수도권1호선_역간거리_20241015.csv"),
    load_distance_data("/content/국가철도공단_수도권2호선_역간거리_20241015.csv"),
    load_distance_data("/content/국가철도공단_수도권3호선_역간거리_20241022.csv"),
    load_distance_data("/content/국가철도공단_수도권4호선_역간거리_20230516.csv"),
    load_distance_data("/content/국가철도공단_수도권5호선_역간거리_20241015.csv"),
]
distance_df = pd.concat(distance_dfs, ignore_index=True)
distance_df["line"] = distance_df["line"].str.extract(r"(\d)").astype(int)
distance_df = distance_df[distance_df["line"].isin([1, 2, 3, 4, 5])]
distance_df["time_min"] = distance_df.apply(
    lambda row: round((row["distance_km"] / speed_dict.get(row["line"], 30)) * 60, 1), axis=1
)

# 연결 정보 예시 만들기
link_data = distance_df.copy()
link_data["link"] = "(" + link_data["station"] + "," + link_data["line"].astype(str) + "):" + link_data["time_min"].astype(str)

# 위경도 정보에서 1~5호선만 필터
latlon_df["호선"] = latlon_df["호선"].astype(int)
final_df = latlon_df[latlon_df["호선"].isin([1, 2, 3, 4, 5])].copy()
final_df["line_num"] = final_df["호선"]
final_df["station_nm"] = final_df["역명"]
final_df["link1"] = ""

# 환승 정보 병합
final_df = final_df.merge(transfer_grouped[["역명", "transfer1", "transfer2"]], on="역명", how="left")
final_df[["transfer1", "transfer2"]] = final_df[["transfer1", "transfer2"]].fillna("")

# 누락된 link2~4 채우기
for col in ["link2", "link3", "link4"]:
    final_df[col] = ""

# 컬럼 정렬 및 결과 출력
input_df = final_df[["line_num", "station_nm", "cx", "cy", "link1", "link2", "link3", "link4", "transfer1", "transfer2"]]
input_df.to_csv("input.csv", index=False, encoding="cp949")

input_df.head(20)


,line_num,station_nm,cx,cy,link1,link2,link3,link4,transfer1,transfer2
0,1,서울,126.972533,37.553150,,,,,,
1,1,시청,126.975407,37.563590,,,,,2,1
2,1,종각,126.983116,37.570203,,,,,,
3,1,종로3가,126.992095,37.570429,,,,,3,5
4,1,종로5가,127.001900,37.570971,,,,,,
5,1,동대문,127.011383,37.571790,,,,,4,1
6,1,동묘앞,127.016459,37.573265,,,,,6,1
7,1,신설동,127.024710,37.576117,,,,,우이신설경전철,2
8,1,제기동,127.034902,37.578116,,,,,,
9,1,청량리,127.045063,37.580148,,,,,경의선,1


In [177]:
def clean_transfer(val):
    try:
        val_int = int(val)
        if 1 <= val_int <= 5: #1호선부터 5호선까지만 출력
            return str(val_int)
    except:
        pass
    return ""

input_df["transfer1"] = input_df["transfer1"].apply(clean_transfer)
input_df["transfer2"] = input_df["transfer2"].apply(clean_transfer)

input_df.to_csv("input.csv", index=False, encoding="cp949")
input_df.head()

<ipython-input-177-232ef9339370>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df["transfer1"] = input_df["transfer1"].apply(clean_transfer)
<ipython-input-177-232ef9339370>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df["transfer2"] = input_df["transfer2"].apply(clean_transfer)


,line_num,station_nm,cx,cy,link1,link2,link3,link4,transfer1,transfer2
0,1,서울,126.972533,37.553150,,,,,,
1,1,시청,126.975407,37.563590,,,,,2,1
2,1,종각,126.983116,37.570203,,,,,,
3,1,종로3가,126.992095,37.570429,,,,,3,5
4,1,종로5가,127.001900,37.570971,,,,,,


In [178]:
# 연결 정보 자동 생성: 각 호선별 정렬 기준으로 인접한 역을 연결하고 link1~4에 입력
from collections import defaultdict

# 호선별로 정렬된 역 리스트 구성
grouped = input_df.sort_values(["line_num", "cy", "cx"]).groupby("line_num")

# 역 연결 관계 저장용 딕셔너리
links_by_node = defaultdict(list)

# 역 이름을 'station_nm_line_num' 형태로 만들어 인접 연결
for line, group in grouped:
    stations = group.reset_index(drop=True)
    for i in range(len(stations) - 1):
        curr = stations.loc[i]
        next_ = stations.loc[i + 1]

        curr_key = f"{curr['station_nm']}_{curr['line_num']}"
        next_key = f"{next_['station_nm']}_{next_['line_num']}"

        # 거리 추정값: 기본 2.0분
        links_by_node[curr_key].append((next_key, 2.0))
        links_by_node[next_key].append((curr_key, 2.0))

# link1~4 채우기
def assign_links(row):
    key = f"{row['station_nm']}_{row['line_num']}"
    links = links_by_node.get(key, [])
    formatted = [f"({k.split('_')[0]},{k.split('_')[1]}):{v}" for k, v in links]
    out = ["", "", "", ""]
    for i in range(min(4, len(formatted))):
        out[i] = formatted[i]
    return pd.Series(out, index=["link1", "link2", "link3", "link4"])

input_df[["link1", "link2", "link3", "link4"]] = input_df.apply(assign_links, axis=1)



input_df.to_csv("input.csv", index=False, encoding="cp949")
input_df.head(20)

<ipython-input-178-0c6702b28581>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df[["link1", "link2", "link3", "link4"]] = input_df.apply(assign_links, axis=1)


,line_num,station_nm,cx,cy,link1,link2,link3,link4,transfer1,transfer2
0,1,서울,126.972533,37.553150,"(시청,1):2.0",,,,,
1,1,시청,126.975407,37.563590,"(서울,1):2.0","(종각,1):2.0",,,2,1
2,1,종각,126.983116,37.570203,"(시청,1):2.0","(종로3가,1):2.0",,,,
3,1,종로3가,126.992095,37.570429,"(종각,1):2.0","(종로5가,1):2.0",,,3,5
4,1,종로5가,127.001900,37.570971,"(종로3가,1):2.0","(동대문,1):2.0",,,,
5,1,동대문,127.011383,37.571790,"(종로5가,1):2.0","(동묘앞,1):2.0",,,4,1
6,1,동묘앞,127.016459,37.573265,"(동대문,1):2.0","(신설동,1):2.0",,,,1
7,1,신설동,127.024710,37.576117,"(동묘앞,1):2.0","(제기동,1):2.0",,,,2
8,1,제기동,127.034902,37.578116,"(신설동,1):2.0","(청량리,1):2.0",,,,
9,1,청량리,127.045063,37.580148,"(제기동,1):2.0",,,,,1


In [179]:
# 기존 transfer1, transfer2 값을 링크 형식으로 다시 구성: (역명,호선):5.0
def transfer_links_like_link(row):
    transfers = []
    for t in ["transfer1", "transfer2"]:
        val = row[t]
        if isinstance(val, str) and "(" in val:
            # 이미 (역명,호선):시간 형식이면 그대로
            transfers.append(val)
        elif isinstance(val, (int, float)) or (isinstance(val, str) and val.isdigit()):
            # 숫자 또는 숫자 문자열이면 포맷팅
            transfers.append(f"({row['station_nm']},{int(val)}):5.0")
        else:
            # 그 외는 빈 칸
            transfers.append("")
    return pd.Series(transfers, index=["transfer1", "transfer2"])


# 적용
final_df[["transfer1", "transfer2"]] = final_df.apply(transfer_links_like_link, axis=1)

final_df.to_csv("input.csv", index=False, encoding="cp949")
final_df.head(20)

,연번,호선,고유역번호(외부역코드),역명,cy,cx,작성일자,노드명,line_num,station_nm,link1,transfer1,transfer2,link2,link3,link4
0,1,1,150,서울,37.553150,126.972533,1974-02-28,서울_1,1,서울,,,,,,
1,2,1,151,시청,37.563590,126.975407,1974-08-15,시청_1,1,시청,,"(시청,2):5.0","(시청,1):5.0",,,
2,3,1,152,종각,37.570203,126.983116,1974-08-15,종각_1,1,종각,,,,,,
3,4,1,153,종로3가,37.570429,126.992095,1974-08-15,종로3가_1,1,종로3가,,"(종로3가,3):5.0","(종로3가,5):5.0",,,
4,5,1,154,종로5가,37.570971,127.001900,1974-03-31,종로5가_1,1,종로5가,,,,,,
5,6,1,155,동대문,37.571790,127.011383,1974-04-30,동대문_1,1,동대문,,"(동대문,4):5.0","(동대문,1):5.0",,,
6,7,1,159,동묘앞,37.573265,127.016459,2005-12-31,동묘앞_1,1,동묘앞,,"(동묘앞,6):5.0","(동묘앞,1):5.0",,,
7,8,1,156,신설동,37.576117,127.024710,1974-03-30,신설동_1,1,신설동,,,"(신설동,2):5.0",,,
8,9,1,157,제기동,37.578116,127.034902,1974-03-30,제기동_1,1,제기동,,,,,,
9,10,1,158,청량리,37.580148,127.045063,1974-08-31,청량리_1,1,청량리,,,"(청량리,1):5.0",,,


In [180]:
# 업로드된 역간 거리 기반 파일들을 다시 사용해 정확한 노선 연결 순서 구성
# link 정보를 거리 기반으로 설정

# 거리 데이터셋들을 불러와서 연결
distance_files = [
    "/content/국가철도공단_수도권1호선_역간거리_20241015.csv",
    "/content/국가철도공단_수도권2호선_역간거리_20241015.csv",
    "/content/국가철도공단_수도권3호선_역간거리_20241022.csv",
    "/content/국가철도공단_수도권4호선_역간거리_20230516.csv",
    "/content/국가철도공단_수도권5호선_역간거리_20241015.csv",
]

# 다양한 컬럼명 케이스에 대응
def load_distance_file(filepath):
    df = pd.read_csv(filepath, encoding="cp949")
    for colname in ["역간거리(km)", "역간 거리(km)", "역간거리"]:
        if colname in df.columns:
            df = df[["선명", "역명", colname]]
            df.columns = ["line", "station", "distance_km"]
            return df
    raise ValueError(f"{filepath}에서 유효한 거리 컬럼을 찾을 수 없습니다.")

distance_df = pd.concat([load_distance_file(f) for f in distance_files], ignore_index=True)
distance_df["line"] = distance_df["line"].str.extract(r"(\d+)").astype(int)
distance_df = distance_df[distance_df["line"].isin([1, 2, 3, 4, 5])]

# 표정속도 정보 로드
speed_df = pd.read_csv("/content/서울교통공사_1_8호선 표정속도 정보_20250314.csv", encoding="cp949")
speed_df = speed_df[speed_df["호선"].apply(lambda x: str(x).isdigit())]
speed_df["호선"] = speed_df["호선"].astype(int)
speed_dict = speed_df.groupby("호선")["표정속도"].mean().to_dict()

# 거리 기반 시간 계산
distance_df["time_min"] = distance_df.apply(
    lambda row: round((row["distance_km"] / speed_dict.get(row["line"], 30)) * 60, 1),
    axis=1
)

# 연결정보 생성: 인접역 순서대로 (역A,역B) → time 형태 저장
link_dict = defaultdict(list)
for i in range(len(distance_df) - 1):
    curr = distance_df.iloc[i]
    next_ = distance_df.iloc[i + 1]
    if curr["line"] == next_["line"]:
        a = f"{curr['station']}_{curr['line']}"
        b = f"{next_['station']}_{next_['line']}"
        t = next_["time_min"]
        link_dict[a].append((b, t))
        link_dict[b].append((a, t))

# final_df 기준으로 적용
def assign_links_from_distance(row):
    key = f"{row['station_nm']}_{row['line_num']}"
    links = link_dict.get(key, [])
    formatted = [f"({k.split('_')[0]},{k.split('_')[1]}):{v}" for k, v in links]
    out = ["", "", "", ""]
    for i in range(min(4, len(formatted))):
        out[i] = formatted[i]
    return pd.Series(out, index=["link1", "link2", "link3", "link4"])

final_df[["link1", "link2", "link3", "link4"]] = final_df.apply(assign_links_from_distance, axis=1)
# 사용자가 요청한 대로 line_num, station_nm, cx, cy, link1~4, transfer1~2 포함
final_full_df = final_df[["line_num", "station_nm", "cx", "cy", "link1", "link2", "link3", "link4", "transfer1", "transfer2"]].copy()

final_full_df.to_csv("input.csv", index=False, encoding="cp949")
final_full_df.head(20)

,line_num,station_nm,cx,cy,link1,link2,link3,link4,transfer1,transfer2
0,1,서울,126.972533,37.553150,,,,,,
1,1,시청,126.975407,37.563590,"(종각,1):2.3","(서울역,1):2.5",,,"(시청,2):5.0","(시청,1):5.0"
2,1,종각,126.983116,37.570203,"(종로3가,1):1.8","(시청,1):2.3",,,,
3,1,종로3가,126.992095,37.570429,"(종로5가,1):2.1","(종각,1):1.8",,,"(종로3가,3):5.0","(종로3가,5):5.0"
4,1,종로5가,127.001900,37.570971,"(동대문,1):1.8","(종로3가,1):2.1",,,,
5,1,동대문,127.011383,37.571790,"(동묘앞,1):1.4","(종로5가,1):1.8",,,"(동대문,4):5.0","(동대문,1):5.0"
6,1,동묘앞,127.016459,37.573265,"(신설동,1):1.6","(동대문,1):1.4",,,"(동묘앞,6):5.0","(동묘앞,1):5.0"
7,1,신설동,127.024710,37.576117,"(회기,1):2.1","(동묘앞,1):1.6",,,,"(신설동,2):5.0"
8,1,제기동,127.034902,37.578116,"(외대앞,1):2.3","(회기,1):3.2",,,,
9,1,청량리,127.045063,37.580148,,,,,,"(청량리,1):5.0"


In [181]:
# 중복 환승 제거 + 시간 형식 수정

def clean_transfer_field(field_list):
    seen = set()
    cleaned = []
    for field in field_list:
        if not isinstance(field, str) or field.strip() == "":
            continue
        try:
            content, time_str = field.split(":")
            time_val = round(float(time_str), 2)
            if content not in seen:
                seen.add(content)
                cleaned.append(f"{content}:{time_val}")
        except:
            continue
    cleaned += [""] * (2 - len(cleaned))
    return pd.Series(cleaned[:2], index=["transfer1", "transfer2"])

# 적용
final_full_df[["transfer1", "transfer2"]] = final_full_df[["transfer1", "transfer2"]].apply(
    lambda row: clean_transfer_field(row.values), axis=1
)



final_full_df.to_csv("input.csv", index=False, encoding="cp949")
final_full_df.head(20)

,line_num,station_nm,cx,cy,link1,link2,link3,link4,transfer1,transfer2
0,1,서울,126.972533,37.553150,,,,,,
1,1,시청,126.975407,37.563590,"(종각,1):2.3","(서울역,1):2.5",,,"(시청,2):5.0","(시청,1):5.0"
2,1,종각,126.983116,37.570203,"(종로3가,1):1.8","(시청,1):2.3",,,,
3,1,종로3가,126.992095,37.570429,"(종로5가,1):2.1","(종각,1):1.8",,,"(종로3가,3):5.0","(종로3가,5):5.0"
4,1,종로5가,127.001900,37.570971,"(동대문,1):1.8","(종로3가,1):2.1",,,,
5,1,동대문,127.011383,37.571790,"(동묘앞,1):1.4","(종로5가,1):1.8",,,"(동대문,4):5.0","(동대문,1):5.0"
6,1,동묘앞,127.016459,37.573265,"(신설동,1):1.6","(동대문,1):1.4",,,"(동묘앞,6):5.0","(동묘앞,1):5.0"
7,1,신설동,127.024710,37.576117,"(회기,1):2.1","(동묘앞,1):1.6",,,"(신설동,2):5.0",
8,1,제기동,127.034902,37.578116,"(외대앞,1):2.3","(회기,1):3.2",,,,
9,1,청량리,127.045063,37.580148,,,,,"(청량리,1):5.0",


In [182]:
import os

# 사용자가 업로드한 국가철도공단 역간거리 파일들 (1~5호선)
korail_files = [
    "/content/국가철도공단_수도권1호선_역간거리_20241015.csv",
    "/content/국가철도공단_수도권2호선_역간거리_20241015.csv",
    "/content/국가철도공단_수도권3호선_역간거리_20241022.csv",
    "/content/국가철도공단_수도권4호선_역간거리_20230516.csv",
    "/content/국가철도공단_수도권5호선_역간거리_20241015.csv"
]

# 각 파일에서 역명 추출 및 정제
additional_stations = []
for file in korail_files:
    try:
        df = pd.read_csv(file, encoding="cp949")
        col_names = df.columns.tolist()
        # 역명 컬럼 찾기
        from_col = next((c for c in col_names if "역명" in c and "기점" in c), None)
        to_col = next((c for c in col_names if "역명" in c and "종점" in c), None)
        line_col = next((c for c in col_names if "호선" in c), None)

        if from_col and to_col and line_col:
            line_num = int("".join(filter(str.isdigit, str(df[line_col].iloc[0]))))  # 1~5
            for station in pd.concat([df[from_col], df[to_col]]).unique():
                additional_stations.append((line_num, station.strip()))
    except Exception as e:
        continue

# 기존에 포함된 역명 목록
existing_stations = set(zip(final_full_df["line_num"], final_full_df["station_nm"]))

# 추가해야 할 역
new_station_tuples = list(set(additional_stations) - existing_stations)

# 위경도 알 수 없는 경우 NaN 처리
new_df = pd.DataFrame(new_station_tuples, columns=["line_num", "station_nm"])
new_df["cx"] = None
new_df["cy"] = None
for col in ["link1", "link2", "link3", "link4", "transfer1", "transfer2"]:
    new_df[col] = ""

# 병합
updated_df = pd.concat([final_full_df, new_df], ignore_index=True)

updated_df.to_csv("input.csv", index=False, encoding="cp949")
updated_df.head(20)

<ipython-input-182-695c43b62b9b>:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_df = pd.concat([final_full_df, new_df], ignore_index=True)


,line_num,station_nm,cx,cy,link1,link2,link3,link4,transfer1,transfer2
0,1,서울,126.972533,37.553150,,,,,,
1,1,시청,126.975407,37.563590,"(종각,1):2.3","(서울역,1):2.5",,,"(시청,2):5.0","(시청,1):5.0"
2,1,종각,126.983116,37.570203,"(종로3가,1):1.8","(시청,1):2.3",,,,
3,1,종로3가,126.992095,37.570429,"(종로5가,1):2.1","(종각,1):1.8",,,"(종로3가,3):5.0","(종로3가,5):5.0"
4,1,종로5가,127.001900,37.570971,"(동대문,1):1.8","(종로3가,1):2.1",,,,
5,1,동대문,127.011383,37.571790,"(동묘앞,1):1.4","(종로5가,1):1.8",,,"(동대문,4):5.0","(동대문,1):5.0"
6,1,동묘앞,127.016459,37.573265,"(신설동,1):1.6","(동대문,1):1.4",,,"(동묘앞,6):5.0","(동묘앞,1):5.0"
7,1,신설동,127.024710,37.576117,"(회기,1):2.1","(동묘앞,1):1.6",,,"(신설동,2):5.0",
8,1,제기동,127.034902,37.578116,"(외대앞,1):2.3","(회기,1):3.2",,,,
9,1,청량리,127.045063,37.580148,,,,,"(청량리,1):5.0",


In [183]:
# 환승 시간 포맷을 (역명,호선):분.초 → (역명,호선):소수점 분 으로 변환

def convert_transfer_format(transfer_cell):
    if pd.isna(transfer_cell) or transfer_cell == "":
        return ""
    try:
        name_part, time_part = transfer_cell.split(":")
        if ":" in time_part:
            # 혹시 시간 형식이 잘못된 경우
            return transfer_cell
        # 시간 문자열을 float으로 변환
        minutes = float(time_part)
        return f"{name_part}:{minutes:.2f}"
    except:
        return transfer_cell

# 변환 적용
final_full_df["transfer1"] = final_full_df["transfer1"].map(convert_transfer_format)
final_full_df["transfer2"] = final_full_df["transfer2"].map(convert_transfer_format)

final_full_df.to_csv("input.csv", index=False, encoding="cp949")

final_full_df.head(20)

,line_num,station_nm,cx,cy,link1,link2,link3,link4,transfer1,transfer2
0,1,서울,126.972533,37.553150,,,,,,
1,1,시청,126.975407,37.563590,"(종각,1):2.3","(서울역,1):2.5",,,"(시청,2):5.00","(시청,1):5.00"
2,1,종각,126.983116,37.570203,"(종로3가,1):1.8","(시청,1):2.3",,,,
3,1,종로3가,126.992095,37.570429,"(종로5가,1):2.1","(종각,1):1.8",,,"(종로3가,3):5.00","(종로3가,5):5.00"
4,1,종로5가,127.001900,37.570971,"(동대문,1):1.8","(종로3가,1):2.1",,,,
5,1,동대문,127.011383,37.571790,"(동묘앞,1):1.4","(종로5가,1):1.8",,,"(동대문,4):5.00","(동대문,1):5.00"
6,1,동묘앞,127.016459,37.573265,"(신설동,1):1.6","(동대문,1):1.4",,,"(동묘앞,6):5.00","(동묘앞,1):5.00"
7,1,신설동,127.024710,37.576117,"(회기,1):2.1","(동묘앞,1):1.6",,,"(신설동,2):5.00",
8,1,제기동,127.034902,37.578116,"(외대앞,1):2.3","(회기,1):3.2",,,,
9,1,청량리,127.045063,37.580148,,,,,"(청량리,1):5.00",


In [184]:
# transfer1과 transfer2에 대해 ':' 뒤에 오는 숫자 형식을 float 형식의 소수점으로 통일

def normalize_transfer_time(cell):
    if pd.isna(cell) or cell == "":
        return ""
    try:
        # 형식: (역명,호선):5.0 → 분리
        match = re.match(r"\(([^,]+),(\d+)\):([\d.]+)", cell)
        if match:
            station, line, time = match.groups()
            time_float = float(time)
            return f"({station},{line}):{round(time_float, 2)}"
    except:
        return cell
    return cell

# 적용
final_full_df["transfer1"] = final_full_df["transfer1"].map(normalize_transfer_time)
final_full_df["transfer2"] = final_full_df["transfer2"].map(normalize_transfer_time)

final_full_df.to_csv("input.csv", index=False, encoding="cp949")

final_full_df.head(20)

,line_num,station_nm,cx,cy,link1,link2,link3,link4,transfer1,transfer2
0,1,서울,126.972533,37.553150,,,,,,
1,1,시청,126.975407,37.563590,"(종각,1):2.3","(서울역,1):2.5",,,"(시청,2):5.0","(시청,1):5.0"
2,1,종각,126.983116,37.570203,"(종로3가,1):1.8","(시청,1):2.3",,,,
3,1,종로3가,126.992095,37.570429,"(종로5가,1):2.1","(종각,1):1.8",,,"(종로3가,3):5.0","(종로3가,5):5.0"
4,1,종로5가,127.001900,37.570971,"(동대문,1):1.8","(종로3가,1):2.1",,,,
5,1,동대문,127.011383,37.571790,"(동묘앞,1):1.4","(종로5가,1):1.8",,,"(동대문,4):5.0","(동대문,1):5.0"
6,1,동묘앞,127.016459,37.573265,"(신설동,1):1.6","(동대문,1):1.4",,,"(동묘앞,6):5.0","(동묘앞,1):5.0"
7,1,신설동,127.024710,37.576117,"(회기,1):2.1","(동묘앞,1):1.6",,,"(신설동,2):5.0",
8,1,제기동,127.034902,37.578116,"(외대앞,1):2.3","(회기,1):3.2",,,,
9,1,청량리,127.045063,37.580148,,,,,"(청량리,1):5.0",


In [185]:
# transfer1, transfer2의 중복 환승 제거: 동일 역명 + 동일 호선이면 제거
def remove_duplicate_transfer(row):
    line = row["line_num"]
    station = row["station_nm"]
    new_transfers = []

    for col in ["transfer1", "transfer2"]:
        val = row[col]
        if isinstance(val, str) and val.startswith("(") and ":" in val:
            try:
                content, time = val.split(":")
                name_in_val, line_in_val = content.replace("(", "").replace(")", "").split(",")
                if int(line_in_val) != line or name_in_val != station:
                    new_transfers.append(val)
                else:
                    new_transfers.append("")  # 동일 역명 + 동일 호선이면 제거
            except:
                new_transfers.append("")
        else:
            new_transfers.append("")
    return pd.Series(new_transfers, index=["transfer1", "transfer2"])

# 적용
final_full_df[["transfer1", "transfer2"]] = final_full_df.apply(remove_duplicate_transfer, axis=1)

final_full_df.to_csv("input.csv", index=False, encoding="cp949")

final_full_df.head(20)


,line_num,station_nm,cx,cy,link1,link2,link3,link4,transfer1,transfer2
0,1,서울,126.972533,37.553150,,,,,,
1,1,시청,126.975407,37.563590,"(종각,1):2.3","(서울역,1):2.5",,,"(시청,2):5.0",
2,1,종각,126.983116,37.570203,"(종로3가,1):1.8","(시청,1):2.3",,,,
3,1,종로3가,126.992095,37.570429,"(종로5가,1):2.1","(종각,1):1.8",,,"(종로3가,3):5.0","(종로3가,5):5.0"
4,1,종로5가,127.001900,37.570971,"(동대문,1):1.8","(종로3가,1):2.1",,,,
5,1,동대문,127.011383,37.571790,"(동묘앞,1):1.4","(종로5가,1):1.8",,,"(동대문,4):5.0",
6,1,동묘앞,127.016459,37.573265,"(신설동,1):1.6","(동대문,1):1.4",,,"(동묘앞,6):5.0",
7,1,신설동,127.024710,37.576117,"(회기,1):2.1","(동묘앞,1):1.6",,,"(신설동,2):5.0",
8,1,제기동,127.034902,37.578116,"(외대앞,1):2.3","(회기,1):3.2",,,,
9,1,청량리,127.045063,37.580148,,,,,,
